In [1]:
from libs.plugin_converter.semantic_kernel_v0_to_openai_function import generate_callables, generate_definitions, make_context
from libs.plugins.plugin_capital import PluginCapital
from libs.plugins.plugin_salesforce import PluginSalesforce
from libs.plugins.plugin_sap import PluginSAP
from libs.plugins.plugin_capivara import PluginCapivaraAlert
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool

import time
import os
from libs.utils.connector_llm import *
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool
from libs.plugin_orchestrator.implementation_tool import OrchestratorWithTool
from libs.plugin_orchestrator.implementation_bare import OrchestratorBare

import dotenv
dotenv.load_dotenv()

True

In [2]:
#semantic_kernel_v0_to_openai_function.generate_callables([(PluginSAP(), "PluginSAP")])

In [1]:
!pip install tabulate

In [4]:
# Bare openai
llm = factory_create_connector_llm(
    provider='azure_openai',
    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
    credentials=CredentialsOpenAI(
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    ),
    hyperparameters={'max_tokens': 2048, 'tool_choice': 'none'}
)

# Tool openai
#llm = factory_create_connector_llm(
#    provider='azure_openai',
#    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
#    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
#    credentials=CredentialsOpenAI(
#        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
#        api_key=os.getenv("AZURE_OPENAI_KEY"),
#    ),
#)


selected_alert = {'id': 'dc2a',
 'title': 'Defective product',
 'description': 'I ordered a Handstückschlauch, but it came already broken',
 'severity': 'Medium'}

plugins = [
    (PluginCapivaraAlert(capivara_base_url='http://capivara:80', alert_id=selected_alert['id']), "PluginServiceNow"),
    (PluginSalesforce(), "PluginSalesforce"),
    (PluginSAP(), "PluginSAP"),
]
orchestrator = OrchestratorBare(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions(plugins)),
    tool_callables = generate_callables(plugins),
    token_limit_input = 2048,
    token_limit_output = None,
    max_steps_recommended = 4,
    max_steps_allowed = 5,
    prompt_app_system=(
        'You are an AI assistant whose goal is to help the use handle a support ticket.\n'
        'Here is some info about this ticket:\n'
        f'Title: {selected_alert["title"]}\n'
        f'Description: {selected_alert["description"]}\n'
        f'Severity: {selected_alert["severity"]}\n'
        'Answer the user questions and help the user to resolve this ticket.\n'
    ),
    #prompt_app_user='hi =D'  # chitchat (don't work well)
    #prompt_app_user='What purchases have this customer done in the past?',  # SAP
    #prompt_app_user='place an order for 10 gloves',  # SAP
    #prompt_app_user='If the customer had purchased any glove in the past, please place an order for 100 of that exact glove model',  # SAP
    #prompt_app_user='IF the customer purchased any hand desinfectant in the past, create an appointment for a sales call and describing we may want to do a follow up sales',  # SAP
    #prompt_app_user='what is the full name of this customer?',  # Salesforce
    #prompt_app_user='create an appointment asking an employee to visit the customer facility, and don\'t forget to mention the customer address',  # Salesforce, SAP
    #prompt_app_user='close this ticket',  # Capivara
    #prompt_app_user='place an order for 10 gloves, then close the current ticket',  # SAP, capivara
    prompt_app_user='If the customer had purchased any glove in the past, please place an order for 100 of that exact glove model and close the current ticket',  # SAP, capivara
)

'''
r = await orchestrator.run_async()
#r = orchestrator.run()
print(r)
print('\n\n' + '-'*30 + '\n\n')
assert type(r.answer) == str
assert len(r.answer) > 4
print(r.answer)
''';

#'''
r_stream = orchestrator.run_stream_async()
chunks = []
async for chunk in r_stream:
    #print(chunk.answer, '\t\t| ', len(chunk.intermediate_results or []))
    print(chunk)
    chunks.append(chunk)
    #time.sleep(0.1)
#''';

'''
r_stream = orchestrator.run_stream()
chunks = []
for chunk in r_stream:
    #print(chunk)
    chunks.append(chunk)
    print(chunk.answer, '\t\t| ', len(chunk.intermediate_results or []))
    time.sleep(0.1)
''';

THIS IS WHAT THE LLM IS TRYING TO DO:
content='{"thought": "I need to check the customer\'s past purchase orders to see if they have bought any gloves before.", "action_name": "PluginSAP_get_purchase_orders", "args": {}}' role='assistant' tool_calls=[]
[2024-08-26 11:59:54 +0000] libs.libs.plugin_orchestrator INFO     Function calling detected by: custom implementation
[2024-08-26 11:59:54 +0000] libs.libs.plugin_orchestrator INFO     At step 1 the function PluginSAP_get_purchase_orders was requested, with args = {}
answer=None citations=None visualizations=None intermediate_results=[IntermediateResult(step=1, status_code=200, function_name='PluginSAP_get_purchase_orders', function_arguments={}, result='[{"deliveryDate": "2021-01-01", "totalAmount": 10000, "items": [{"materialId": "IPS Natural Die Material ND1", "quantity": 1}, {"materialId": "Untersuchungshandschuhe Nitril light lemon Gr. M", "quantity": 10}, {"materialId": "Antiseptica r.f.u. H\\u00e4ndedesinfektion Flasche 1 Liter",

In [6]:
#############################################
# Dataset generation
#orchestrator._full_message_history_for_debugging
# This is the expected chat-over-text format:
# <|system|> You are a chatbot who can help code!</s> <|user|> Write me a function to calculate the first 10 digits of the fibonacci sequence in Python and print it out to the CLI.</s> <|assistant|>
# Before finetunning, we have to convert
# this format is called "Zephyr", because it is used in the model HuggingFaceH4/zephyr-7b-alpha

In [2]:
from libs.plugin_converter.semantic_kernel_v0_to_openai_function import generate_callables, generate_definitions, make_context
from libs.plugins.plugin_capital import PluginCapital
from libs.plugins.plugin_salesforce import PluginSalesforce
from libs.plugins.plugin_sap import PluginSAP
from libs.plugins.plugin_capivara import PluginCapivaraAlert
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool

import time
import os
from libs.utils.connector_llm import *
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool
from libs.plugin_orchestrator.implementation_tool import OrchestratorWithTool
from libs.plugin_orchestrator.implementation_bare import OrchestratorBare
from libs.plugin_orchestrator.implementation_bare_finetuned import OrchestratorBareFinetuned
from libs.utils.json_resilient import *

import dotenv
dotenv.load_dotenv()

True

In [3]:
# Bare llamacpp
#'''
llm = factory_create_connector_llm(
    provider='llama_cpp',
    modelname='not-needed',
    version='not-needed',
    credentials=CredentialsOpenAI(
        base_url='http://llm-server:8080/v1',
        api_key='not-needed',
    ),
    hyperparameters={'max_tokens': 150}
)

plugins = [
    (PluginCapital(), "PluginCapital"),
]
orchestrator = OrchestratorBareFinetuned(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions(plugins)),
    tool_callables = generate_callables(plugins),
    token_limit_input = 1024,
    token_limit_output = None,
    max_steps_recommended = 2,
    max_steps_allowed = 2,
    prompt_app_system='You are an AI assistant whose goal is to answer the user question.',
     prompt_app_user='What is the capital of france?',
)

r_stream = orchestrator.run_stream_async()
chunks = []
async for chunk in r_stream:
    print(chunk.answer, '\t\t| ', len(chunk.intermediate_results or []))
    #print(chunk)
    chunks.append(chunk)
    #time.sleep(0.1)
#''';

THIS IS WHAT THE LLM IS TRYING TO DO:
content="```\nconst { PluginCapital_get_capital } = require('plugin-capital');\n\nconst country = 'French Republic';\n\nconst capital = await PluginCapital_get_capital(country);\n\nconsole.log(capital); // Output: 'Paris'\n```\n\nIn this example, the `PluginCapital_get_capital` function is called with the country as a parameter. The function is expected to return the name of the capital of the French Republic.\n\nStep 2: Answer the user's question\nAfter executing the `PluginCapital_get_capital` function, the function's output is passed to the `answer" role='assistant' tool_calls=[]
[2024-08-27 06:48:54 +0000] libs.libs.plugin_orchestrator INFO     Function calling detected by: custom implementation
[2024-08-27 06:48:54 +0000] libs.utils.json_resilient WARNING  json_loads_resilient: received jsonl with multiple elemnts, returning only the first valid element...
[2024-08-27 06:48:54 +0000] libs.libs.plugin_orchestrator WARNING  Could not parse, erro

MaxStepsExceededError: Exceeded the maximum number of steps allowed: 2